# Export Network Set as Generalized MDIF File

Frequently a set of `Networks` is recorded while changing some other parameters; like temperature, voltage, current, etc. Once this set of data acquired, it is sometime usefull to combine all the networks into a single Generalized MDIF file for use in CAD tools like AWR Microwave Office.

In [1]:
import skrf as rf 
import numpy as np
import tempfile
import zipfile
import pathlib
import requests

## Narda 3752 phase shifter

In this example, we are characterizing an old [narda phase shifter 3752](https://nardamiteq.com/docs/119-PHASESHIFTERS.PDF) at 1.5 GHz. 


![narda 3752 phase shifter](phase_shifter_measurements/Narda_3752.jpg)

In order to deduce the phase shift that one can obtain at this specific frequency, we have measured scattering parameters in the 1-2 GHz band at 19 positions of the phase knob (from 0 to 180). These measurements are loaded into a [NetworkSets](../../tutorials/NetworkSet.ipynb) object:

In [2]:
# Array containing the 19 phase shift indicator values
indicators_mes = np.linspace(0, 180, num=19)  # from 0 to 180 per 10

In [3]:
ntw_set = rf.NetworkSet.from_zip('phase_shifter_measurements/phase_shifter_measurements.zip')
print('ntw_set contains', len(ntw_set), 'networks')

ntw_set contains 19 networks


Make "indicator" an MDIF variable of type "double" and save the NetworkSet to "phase_shifter.mdif".

In [4]:
values = {"indicator": indicators_mes}
datatypes = {"indicator": "double"}
ntw_set.write_mdif("phase_shifter.mdif",values,datatypes)

## ADRF5720

[ADRF5720](https://www.analog.com/en/products/adrf5720.html) is 6-bit 9 kHz to 40 GHz digital step attenuator and is measured over temperature which results in many Touchstone files.

In [5]:
# download the zip archive of ADRF5720 Touchstone files
url = "https://www.analog.com/media/en/simulation-models/s-parameters/ADRF5720_Sparameters.zip"
response = requests.get(url)
open("ADRF5720.zip", "wb").write(response.content)
tmpdir = pathlib.Path(tempfile.mkdtemp())
zf = zipfile.ZipFile("ADRF5720.zip")
zf.extractall(path = tmpdir)
zf.close()

In [10]:
# filter out one file (which contains '5720_noDC')
input_files = [file for file in tmpdir.rglob('*.s2p') if not '5720_noDC' in file.stem]
ns = rf.NetworkSet(rf.read_all(files=[str(file) for file in input_files]))
print('ns contains', len(ns), 'networks')

ns contains 256 networks


In [7]:
# extract the attenuation value from the filenames and store in list
attn = []
temp = []
for f in input_files:
    _,_,_,_,_,a,t = f.stem.split('_')
    t = t.replace('M','-').replace('C','')
    attn.append(float(a))
    temp.append(int(t))
    
# sort files
v = list(zip(attn,temp,input_files))
v.sort()
(attn,temp,input_files) = list(zip(*v))

values = {'ATTEN': attn, 'TEMP_C': temp }
datatypes = {'ATTEN': 'double', 'TEMP_C': 'double'}

In [8]:
# write to a generalized MDIF file
ns.write_mdif("ADRF5720.mdif",values,datatypes)


Finally, the parameterized MDIF files can be imported into AWR Microwave Office:

![AWR](import_mdif_to_awr_mwo.png)